In [1]:
import pandas as pd
import pickle

In [2]:
################################################
################################################
################################################
################################################
# USER DATA #
################################################
################################################
################################################
################################################

In [3]:
## Original JSON files are not formatted ###
## Formatted with process_date.py ##
## Loading newly formatted data ##
with open("/Users/evro/Documents/code/python/fetch/data/cleaned/cleaned_users.json") as f:
    users_data = pd.read_json(f)
users = pd.DataFrame(users_data)

In [4]:
## Reviewing data
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
1,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
2,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI
3,{'$oid': '5ff1e1eacfcf6c399c274ae6'},True,{'$date': 1609687530554},{'$date': 1609687530597},consumer,Email,WI
4,{'$oid': '5ff1e194b6a9d73a3a9f1052'},True,{'$date': 1609687444800},{'$date': 1609687537858},consumer,Email,WI


In [5]:
users.dtypes

_id             object
active            bool
createdDate     object
lastLogin       object
role            object
signUpSource    object
state           object
dtype: object

In [6]:
######################## USER DATA NORMALIZE & CLEANING ########################

In [7]:
# Convert user id to string
users["_id"] = users["_id"].apply(lambda x: x.get("$oid", str(x)))

In [8]:
# Convert createdDate to string
users["createdDate"] = users["createdDate"].apply(lambda x: x.get("$date", str(x)))

In [9]:
# Tried running a conversion for lastLogin but received errors, so I'm investigating to see data types for last login
users["lastLogin"].apply(type).value_counts()

lastLogin
<class 'dict'>     433
<class 'float'>     62
Name: count, dtype: int64

In [10]:
# Let's see which are floats in the lastLogin column
users[users["lastLogin"].apply(lambda x: isinstance(x, float))]

,_id,active,createdDate,lastLogin,role,signUpSource,state
97,5ff616a68f142f11dd189163,True,1609963174996,NaN,consumer,Email,KY
143,5ffe115404929101d0aaebb2,True,1610486100208,NaN,consumer,Email,AL
148,5ffe115404929101d0aaebb2,True,1610486100208,NaN,consumer,Email,AL
170,5e27526d0bdb6a138c32b556,True,1579635309795,NaN,consumer,Google,WI
180,6002475cfb296c121a81b98d,True,1610762076571,NaN,consumer,Email,WI
...,...,...,...,...,...,...,...
381,60186237c8b50e11d8454d5f,True,1612210743551,NaN,consumer,Email,NaN
382,60186237c8b50e11d8454d5f,True,1612210743551,NaN,consumer,Email,NaN
389,60217fa799409b11fcf899fe,True,1612808103714,NaN,consumer,Email,WI
420,5fb0a078be5fc9775c1f3945,True,1605410936818,NaN,consumer,Google,AL


In [11]:
# Get the value of "$date" if lastLogin is a dict otherwise return value
users["lastLogin"] = users["lastLogin"].apply(lambda x: x.get("$date") if isinstance(x, dict) else x)

In [12]:
# review data after clean
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,Email,WI


In [13]:
# Format/clean createdDate and lastLogin to readable and standard formats
users[["createdDate", "lastLogin"]] = users[["createdDate", "lastLogin"]].applymap(lambda x: pd.to_datetime(x.get("$date") if isinstance(x, dict) else x, unit='ms', errors='coerce', utc=True))

/var/folders/cp/y_wm33cd463fshnp8b9k5q640000gn/T/ipykernel_88288/3548197781.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  users[["createdDate", "lastLogin"]] = users[["createdDate", "lastLogin"]].applymap(lambda x: pd.to_datetime(x.get("$date") if isinstance(x, dict) else x, unit='ms', errors='coerce', utc=True))


In [14]:
## Review after cleaned dates
users.head()

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI


In [15]:
# Check data types
users.dtypes

_id                          object
active                         bool
createdDate     datetime64[ns, UTC]
lastLogin       datetime64[ns, UTC]
role                         object
signUpSource                 object
state                        object
dtype: object

In [16]:
# Convert objects to strings
users = users.astype({col: "string" for col in users.select_dtypes(include=["object"]).columns})

In [17]:
users.dtypes

_id                  string[python]
active                         bool
createdDate     datetime64[ns, UTC]
lastLogin       datetime64[ns, UTC]
role                 string[python]
signUpSource         string[python]
state                string[python]
dtype: object

In [18]:
users.head()
# Why are there bullets in the active bool?

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI


In [19]:
######################## END USER DATA CLEANING/NORMALIZATION ########################

In [20]:
######################## VALIDATE USER DATA ########################

In [21]:
# Are there null values?
users.isnull().sum()

_id              0
active           0
createdDate      0
lastLogin       62
role             0
signUpSource    48
state           56
dtype: int64

In [22]:
# Ensure only True/False values exist.
users['active'].value_counts()

active
True     494
False      1
Name: count, dtype: int64

In [23]:
# Which roles are there?
users["role"].value_counts()

role
consumer       413
fetch-staff     82
Name: count, dtype: Int64

In [24]:
# Which users do not have a state?
users[users["state"].isnull()]

,_id,active,createdDate,lastLogin,role,signUpSource,state
344,60145ff384231211ce796d51,True,2021-01-29 19:20:19.722000+00:00,NaT,consumer,Email,<NA>
350,60145ff384231211ce796d51,True,2021-01-29 19:20:19.722000+00:00,NaT,consumer,Email,<NA>
375,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
376,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
378,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
422,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>
423,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>
424,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>


In [25]:
# Ensure only True/False values exist
users['active'].value_counts()

active
True     494
False      1
Name: count, dtype: int64

In [26]:
######################## END VALIDATE USER DATA ########################

In [27]:
######################## DETECT USER ISSUES ########################

In [28]:
# Check for duplicate user records where all column values are identical.
users[users.duplicated()].sort_values(by="_id")

,_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
478,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
479,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
...,...,...,...,...,...,...,...
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
373,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
374,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
387,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI


In [29]:
# How many dups do we have of users and last login?
users.duplicated(subset=["_id", "lastLogin"]).sum()

283

In [30]:
# Check for duplicate user records based on user ID and last login timestamp.
# This helps identify multiple entries for the same user with the same last login time. Data dups?
users[users.duplicated(subset=["_id", "lastLogin"], keep=False)].sort_values("lastLogin")

,_id,active,createdDate,lastLogin,role,signUpSource,state
18,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
13,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
11,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
...,...,...,...,...,...,...,...
378,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
420,5fb0a078be5fc9775c1f3945,True,2020-11-15 03:28:56.818000+00:00,NaT,consumer,Google,AL


In [31]:
# Find users with same last login. May hint towards possible abuse or bots.
users[users.duplicated(subset=["lastLogin"], keep=False)].sort_values("lastLogin")

,_id,active,createdDate,lastLogin,role,signUpSource,state
18,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
13,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
11,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30.554000+00:00,2021-01-03 15:25:30.597000+00:00,consumer,Email,WI
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04.800000+00:00,2021-01-03 15:25:37.858000+00:00,consumer,Email,WI
...,...,...,...,...,...,...,...
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
389,60217fa799409b11fcf899fe,True,2021-02-08 18:15:03.714000+00:00,NaT,consumer,Email,WI
420,5fb0a078be5fc9775c1f3945,True,2020-11-15 03:28:56.818000+00:00,NaT,consumer,Google,AL


In [32]:
# Let's find user id dups with dup dates
users[users.duplicated(subset=["_id"], keep=False)].sort_values("_id")

,_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
478,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
...,...,...,...,...,...,...,...
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
387,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
385,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
394,60229990b57b8a12187fe9e0,True,2021-02-09 14:17:52.581000+00:00,2021-02-09 14:17:52.626000+00:00,consumer,Email,WI


In [33]:
# Can we detect abnormal activity? If multiple users have same state, source, and id it could indicate abuse.
users.groupby(["state", "signUpSource"])["_id"].count().sort_values(ascending=False)

state  signUpSource
WI     Email           376
NH     Email            20
AL     Email             9
OH     Email             5
AL     Google            3
IL     Email             2
CO     Email             1
KY     Email             1
SC     Email             1
WI     Google            1
Name: _id, dtype: int64

In [34]:
# Show states that are not valid
valid_states = {'AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA',
                'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD',
                'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
                'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC',
                'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY'}

users[~users['state'].isin(valid_states)]

,_id,active,createdDate,lastLogin,role,signUpSource,state
344,60145ff384231211ce796d51,True,2021-01-29 19:20:19.722000+00:00,NaT,consumer,Email,<NA>
350,60145ff384231211ce796d51,True,2021-01-29 19:20:19.722000+00:00,NaT,consumer,Email,<NA>
375,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
376,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
378,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
422,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>
423,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>
424,5a43c08fe4b014fd6b6a0612,True,2017-12-27 15:47:27.059000+00:00,2021-02-12 16:22:37.155000+00:00,consumer,<NA>,<NA>


In [35]:
# Which users do not have Google or Email as source?
expected_sources = {"Google", "Email"}
users[~users["signUpSource"].isin(expected_sources)]

,_id,active,createdDate,lastLogin,role,signUpSource,state
388,55308179e4b0eabd8f99caa2,True,2015-04-17 03:43:53.186000+00:00,2018-05-07 17:23:40.003000+00:00,consumer,<NA>,WI
395,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
396,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
397,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
398,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
399,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
400,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
401,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
402,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI
403,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54.302000+00:00,2021-02-08 16:42:58.117000+00:00,fetch-staff,<NA>,WI


In [36]:
# Which users do not have a last login?
users[users["lastLogin"].isnull()]

,_id,active,createdDate,lastLogin,role,signUpSource,state
97,5ff616a68f142f11dd189163,True,2021-01-06 19:59:34.996000+00:00,NaT,consumer,Email,KY
143,5ffe115404929101d0aaebb2,True,2021-01-12 21:15:00.208000+00:00,NaT,consumer,Email,AL
148,5ffe115404929101d0aaebb2,True,2021-01-12 21:15:00.208000+00:00,NaT,consumer,Email,AL
170,5e27526d0bdb6a138c32b556,True,2020-01-21 19:35:09.795000+00:00,NaT,consumer,Google,WI
180,6002475cfb296c121a81b98d,True,2021-01-16 01:54:36.571000+00:00,NaT,consumer,Email,WI
...,...,...,...,...,...,...,...
381,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
382,60186237c8b50e11d8454d5f,True,2021-02-01 20:19:03.551000+00:00,NaT,consumer,Email,<NA>
389,60217fa799409b11fcf899fe,True,2021-02-08 18:15:03.714000+00:00,NaT,consumer,Email,WI
420,5fb0a078be5fc9775c1f3945,True,2020-11-15 03:28:56.818000+00:00,NaT,consumer,Google,AL


In [37]:
# Detect duplicate user records where both user ID and sign-up source are the same.
# This could indicate users signing up multiple times with the same source.
users[users.duplicated(subset=["_id", "signUpSource"], keep=False)].sort_values("_id")

,_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
478,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
...,...,...,...,...,...,...,...
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
387,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
385,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
394,60229990b57b8a12187fe9e0,True,2021-02-09 14:17:52.581000+00:00,2021-02-09 14:17:52.626000+00:00,consumer,Email,WI


In [38]:
# Identify users who have the same account creation date.
# This could detect bulk sign-ups or potential bots if many users were created at the exact same time.
users[users.duplicated(subset=["createdDate"], keep=False)].sort_values("createdDate")

,_id,active,createdDate,lastLogin,role,signUpSource,state
494,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
475,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
476,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
477,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
478,54943462e4b07e684157a532,True,2014-12-19 14:21:22.381000+00:00,2021-03-05 16:52:23.204000+00:00,fetch-staff,<NA>,<NA>
...,...,...,...,...,...,...,...
365,60189c94c8b50e11d8454f6b,True,2021-02-02 00:28:04.020000+00:00,2021-02-02 00:28:04.073000+00:00,consumer,Email,WI
387,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
385,601c2c05969c0b11f7d0b097,True,2021-02-04 17:16:53.700000+00:00,2021-02-04 17:20:30.228000+00:00,consumer,Email,WI
394,60229990b57b8a12187fe9e0,True,2021-02-09 14:17:52.581000+00:00,2021-02-09 14:17:52.626000+00:00,consumer,Email,WI
